In [1]:
from scipy import stats
import pandas as pd
from adjustText import adjust_text
import statsmodels.stats.proportion as ssp
import statsmodels.api as sm
import sys
sys.path.append('../figures/scripts/')
import funclib as flib

### A model to predict alpha based on exposure to developmental stages

Load data

In [2]:
directory = "../"

complete_data = pd.read_csv("{}/data/Table_S3.csv".format(directory))

species2common = flib.df2dict(complete_data, "Species", "Common_name")

The model using parameters from evolutionary data

In [15]:
E = 7/3e9

predicted_alphas_l = []

for i,r in complete_data[complete_data.Group=="Mammals"].iterrows():
    
    # Parameters
    p_age, m_age = r.Generation_time_y, r.Generation_time_y
    y_rate = r.MutPerYearUCSC/1e6
    gen_rate = p_age*y_rate
    
    # Predict alpha
    pred_alphas = []
    mu_f, mu_m = flib.get_parental_age_effects(gen_rate, m_age, p_age, E, 4)
    pred_alpha = flib.predict_alpha(mu_f, mu_m, m_age, p_age, E)
    
    # With different p-to-m generation time ratio
    average_age = p_age
    pred_alpha_ratios = {}
    
    for name,ratio in zip(["min", "max"],[0.7, 1.3]):
        
        m_age, p_age = flib.ratioG(average_age, ratio)
        mu_f, mu_m = flib.get_parental_age_effects_yrate(y_rate, m_age, p_age, E, 4)
        pred_alpha_ratios[name] = flib.predict_alpha(mu_f, mu_m, m_age, p_age, E)
        
    # Keep data for dataframe
    predicted_alphas_l.append([r.Species, pred_alpha, pred_alpha_ratios["min"], pred_alpha_ratios["max"]])

# Build dataframe
predicted_alphas = pd.DataFrame(predicted_alphas_l)
predicted_alphas.columns = ["species","predicted_alpha", "predicted_alpha_lwr", "predicted_alpha_upr"]

predicted_alphas["alpha"] = complete_data.Alpha
predicted_alphas["max_alpha"] = complete_data.Alpha_upr
predicted_alphas["min_alpha"] = complete_data.Alpha_lwr

The model using parameters from pedigree data

In [17]:
predicted_alphas_dnms = pd.DataFrame()
cols = ["Species", "Common_name", "Age", "Paternal", "Maternal", "dnm_alpha", "maternal_slope", "paternal_slope", "predicted_alpha_dnms", "total_muts", "trios","maternal_age","paternal_age"]

for i,r in complete_data[complete_data.MutRatePerGenDNMs>0].iterrows():
    
    sp = r.Species
    
    # Skip flycatcher
    if sp=="Ficedula_albicollis":
        continue
        
    # Useful info
    cn = species2common[sp]
    m,p = r.Maternal_dnms, r.Paternal_dnms
    alpha = p/m
    trios = r.n_Trios
    
    # Parameters
    p_age, m_age = r.Paternal_age_y, r.Maternal_age_y
    gen_rate = r.MutRatePerGenDNMs
    
    # Predict alpha
    mu_f, mu_m = flib.get_parental_age_effects(gen_rate, m_age, p_age, E, 4)
    pred_alpha = flib.predict_alpha(mu_f, mu_m, m_age, p_age, E)
    
    # Build dataframe
    c_df = pd.DataFrame([sp, cn, np.mean([p_age,m_age]), p, m, alpha, mu_f, mu_m, pred_alpha, p+m, trios, m_age, p_age]).transpose()
    c_df.columns = cols
    predicted_alphas_dnms = pd.concat([predicted_alphas_dnms,c_df])

Save results

In [19]:
evo_dict = flib.df2dict(predicted_alphas, "species", "predicted_alpha")
dnm_dict = flib.df2dict(predicted_alphas_dnms, "Species", "predicted_alpha_dnms")

Me = int(np.round(E*3e9))
complete_data["Predicted_alpha_evo_Me{}".format(Me)] = [evo_dict[sp] if sp in evo_dict else np.nan for sp in complete_data.Species]
complete_data["Predicted_alpha_dnm_Me{}".format(Me)] = [dnm_dict[sp] if sp in dnm_dict else np.nan for sp in complete_data.Species]

In [36]:
complete_data[[c for c in complete_data.columns if c not in ["Predicted_alpha_dnms","Predicted_alpha_evo"]]].to_csv("../data/Table_S1.csv")